<a href="https://colab.research.google.com/github/Algocrat/slm-dragon-labs/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1 – Check GPU

In [1]:
!nvidia-smi

Mon Aug 11 10:55:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             41W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

Step 2 — Install a matched PyTorch build (auto-detect CUDA)


In [3]:
import os, sys, subprocess, textwrap, json

# Uninstall any preinstalled/conflicting stacks
!pip -q uninstall -y torch torchvision torchaudio fastai
!pip -q cache purge

candidates = [
    ("cu124", None),   # latest for CUDA 12.4
    ("cu121", None),   # latest for CUDA 12.1
    ("cu118", None),   # latest for CUDA 11.8
]

def try_install(tag):
    print(f"\nTrying PyTorch wheels for {tag} ...")
    cmd = f"pip install --no-cache-dir --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/{tag}"
    rc = os.system(cmd)
    if rc != 0:
        print(f"Install attempt for {tag} returned code {rc}.")
        return False
    try:
        import torch
        print("Torch import OK:", torch.__version__)
        print("CUDA available:", torch.cuda.is_available())
        return True
    except Exception as e:
        print("Torch import failed:", e)
        return False

ok = False
for tag, _ in candidates:
    if try_install(tag):
        ok = True
        break

if not ok:
    raise SystemExit("Could not install a matching torch build. Make sure GPU is enabled, then rerun this cell.")



Trying PyTorch wheels for cu124 ...
Torch import OK: 2.6.0+cu124
CUDA available: True


Step 3 – Install core libraries

In [ ]:
!pip install -U unsloth accelerate transformers peft datasets bitsandbytes sentencepiece --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.7 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.9.0
    Uninstalling accelerate-1.9.0:
      Successfully uninstalled accelerate-1.9.0


Step 4 – Sanity check Torch + GPU



In [ ]:
import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

Torch: 2.6.0+cu124
CUDA available: True
Device: NVIDIA A100-SXM4-40GB


Step 5 -  Load a model with Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = torch.float16

model_name = "unsloth/llama-2-7b-bf16"  # fallback: "unsloth/mistral-7b-v0.2-bf16"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model)
print("Model loaded.")


ImportError: Unsloth: Please install unsloth_zoo via `pip install unsloth_zoo`